## Import libraries

In [1]:
from pathlib import Path
import pandas as pd
import math
import openpyxl

# Create peak pressure file - Attempt 1 (without peak index)

## Access input folders

In [44]:
# Get the current working directory
current_dir = Path.cwd()

# Access the desired directory relative to the current working directory
input_dir1 = current_dir.parent.parent / 'LSDYNA3D' / 'RP_results_all' / 'CompB'

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

2 [WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/01.0m00.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/01.0m03.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/01.0m07.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/01.0m11.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/01.0m15.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/01.0m19.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/01.0m23.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/01.0m25.0kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/02.5m00.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/02.5m03.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/02.5m07.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/02.5m11.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/02.5m15.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/CompB/02.5m19.5kg'), WindowsPath('G:/Chamodi/LSDYNA3

## Dataframe creation

In [45]:
# Define the empty dataframe
df = pd.DataFrame(columns = ['Explosive type',
                             'Explosive mass',
                             'Perpendicular standoff distance',
                             'Incident angle',
                             'Peak reflected pressure'])

# Access each folder inside input_dir2
for folder in input_dir2:
    #print ("folder", folder)
    
    # obtain explosive mass and perpendicular standoff distance
    p_sd = float(folder.name.split("m")[0])
    e_mass = float(folder.name.split("m")[1].split("kg")[0])
    #print (p_sd,e_mass)
    
    # Make a list of data file names, ignoring files named 'trhist'
    files = [file for file in folder.rglob("*") if not file.name == 'trhist']
        
    # Access each file in the list 
    for file in files:
        #print ("file", file)
        
        #read the incident angle
        if file.stem == '1':
            i_angle = 0
        elif file.stem == '2':
            i_angle = 15
        elif file.stem == '3':
            i_angle = 30
        elif file.stem == '4':
            i_angle = 45
        elif file.stem == '5':
            i_angle = 60
        else:
            continue
        
        # Open the text file
        with open(file, 'r') as filetxt:
            
            # Read the 4th row (index 3, 0-based indexing)
            row4 = filetxt.readlines()[3]
            filetxt.seek(0)
            #print (row4)

            # Check if "Pressure" is in the 4th row
            if "Pressure" in row4:
                # Read the 8th row (index 7, 0-based indexing)
                row8 = filetxt.readlines()[7]
                #print (row8)
                
                # Split the row into parts based on whitespace
                parts = row8.split()
                
                # read peak reflected pressure
                pr_pressure = float(parts[2])
                #print (pr_pressure)
                
                # append rows to an empty DataFrame
                new_row = {'Explosive type': ['Composition B'],
                           'Explosive mass' :[e_mass],
                           'Perpendicular standoff distance' : [p_sd],
                           'Incident angle' : i_angle,
                           'Peak reflected pressure' : [pr_pressure/1000]}
                new_row_data = pd.DataFrame(new_row)
                df = pd.concat([df, new_row_data], ignore_index = True)
            else:
                error_message = f'Error: {row4}, {p_sd}, {e_mass}'
                raise ValueError(error_message)
           
print (df)     

    Explosive type  Explosive mass  Perpendicular standoff distance  \
0    Composition B             0.5                              1.0   
1    Composition B             0.5                              1.0   
2    Composition B             0.5                              1.0   
3    Composition B             0.5                              1.0   
4    Composition B             0.5                              1.0   
..             ...             ...                              ...   
435  Composition B            25.0                             20.0   
436  Composition B            25.0                             20.0   
437  Composition B            25.0                             20.0   
438  Composition B            25.0                             20.0   
439  Composition B            25.0                             20.0   

    Incident angle  Peak reflected pressure  
0                0                 1126.810  
1               15                  681.880  
2        

## Create peak pressure excel file

In [46]:
with pd.ExcelWriter('Peak_Reflected_Pressure_CompB.xlsx', engine = "openpyxl", mode='a', if_sheet_exists='replace') as writer:  
    df.to_excel(writer, sheet_name = "Sheet1", index = False, header = True)